# ATL06 + Raster Sampling

A quick demo using pangeo tools

In [ ]:
# NOTE: Need intake-stac master 
#!pip install git+https://github.com/intake/intake-stac@master
import intake
import satsearch

import pandas as pd
import geopandas as gpd
import xarray as xr

import hvplot.xarray
import hvplot.pandas

In [ ]:
# Grab a raster DEM
# (min lon, min lat, max lon, max lat)
mt_rainier_bbox = (-121.849, 46.799, -121.660, 46.904)

URL='https://cmr.earthdata.nasa.gov/cmr-stac/LPDAAC_ECS'
results = satsearch.Search.search(url=URL,
                                  collections=['C204582034-LPDAAC_ECS'], #SRTMGL3
                                  bbox=mt_rainier_bbox,    
                                 )

items = results.items()
print('%s items' % len(items))
list(items)

In [ ]:
# print out full JSON metadata
#item._stac_obj._data

In [ ]:
catalog = intake.open_stac_item_collection(items)
item = catalog['G205277334-LPDAAC_ECS']
img = item.browse.plot()
img

In [ ]:
# Can't yet read hgt.zip directly, so just download locally and then open
# NOTE: need a .netrc file with your earthdata login for this
localFile = item._stac_obj.download('data')
print(localFile)

In [ ]:
# Rasterio can open zip files
import rasterio
with rasterio.open(f'zip://{localFile}!N46W122.hgt') as src:
    print(src.profile)

In [ ]:
# So can Xarray
da = xr.open_rasterio(f'zip://{localFile}!N46W122.hgt')
da

In [ ]:
# Some cleanup - hvplot doesn't like these units
da = da.squeeze('band')
da.attrs.pop('units')

In [ ]:
img = da.hvplot.image(cmap='gray', data_aspect=1, frame_width=500)
img

In [ ]:
# Grab ATL06
URL='https://cmr.earthdata.nasa.gov/cmr-stac/NSIDC_ECS'
results = satsearch.Search.search(url=URL,
                                  collections=['C1706333750-NSIDC_ECS'], #ATL03
                                  bbox=mt_rainier_bbox,    
                                 )

items = results.items()
print('%s items' % len(items))
list(items)[:5]

In [ ]:
# Just grab the top one
catalog = intake.open_stac_item_collection(items)
item = catalog['G1720880018-NSIDC_ECS']

In [ ]:
# Full JSON metadata
#item._stac_obj._data

In [ ]:
# Can't yet read HDF5 directly (Earthdata login issue), so download and read locally
# NOTE: need a .netrc file with your earthdata login for this
localFile = item._stac_obj.download('data')
print(localFile)

In [ ]:
# Load with xarray
daI = xr.open_dataset(localFile, group='gt1l/land_ice_segments')
daI

In [ ]:
# Sample DEM with IS2 points
# http://xarray.pydata.org/en/stable/interpolation.html#advanced-interpolation
# For simplicity take first 1000 points
df = daI.isel(delta_time=slice(0,1000)).to_dataframe()
df

In [ ]:
df.describe()

In [ ]:
# Plot raster and points with holoviews
gf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs='epsg:4326')
gf.head()

In [ ]:
points = gf.hvplot.points(c='h_li', frame_height=400, data_aspect=1, colorbar=True)

In [ ]:
img * points

In [ ]:
# Sample raster at these points using a variety of techniques
# http://xarray.pydata.org/en/stable/interpolation.html#advanced-interpolation

#method: {'linear', 'nearest'} for multidimensional array,
#    {'linear', 'nearest', 'zero', 'slinear', 'quadratic', 'cubic'} for 1-dimensional array.
    
x = xr.DataArray(df.longitude.values, dims="z")
y = xr.DataArray(df.latitude.values, dims="z")
daI = da.interp(x=x, y=y) #default method is linear

In [ ]:
df['h_srtm'] = daI.values

In [ ]:
df.hvplot.scatter(y=['h_li', 'h_srtm'])